### Optimisation of a Neocortical Layer 5 Pyramidal Cell

This notebook shows you how to optimise the maximal conductance of Neocortical Layer 5 Pyramidal Cell as used in Markram et al. 2015.

Author of this script: Werner Van Geit @ Blue Brain Project

Choice of parameters, protocols and other settings was done by Etay Hay @ HUJI

What's described here is a more advanced use of BluePyOpt. We suggest to first go through the introductary example here: https://github.com/BlueBrain/BluePyOpt/blob/master/examples/simplecell/simplecell.ipynb

If you use the methods in this notebook, we ask you to cite the following publications when publishing your research:

Van Geit, W., M. Gevaert, G. Chindemi, C. Rössert, J.-D. Courcol, E. Muller, F. Schürmann, I. Segev, and H. Markram (2016, March). BluePyOpt: Leveraging open source software and cloud infrastructure to optimise model parameters in neuroscience. ArXiv e-prints. http://arxiv.org/abs/1603.00500

Markram, H., E. Muller, S. Ramaswamy, M. W. Reimann, M. Abdellah, C. A. Sanchez, A. Ailamaki, L. Alonso-Nanclares, N. Antille, S. Arsever, et al. (2015). Reconstruction and simulation of neocortical microcircuitry. Cell 163(2), 456–492. http://www.cell.com/abstract/S0092-8674%2815%2901191-5

Some of the modules loaded in this script are located in the L5PC example folder: https://github.com/BlueBrain/BluePyOpt/tree/master/examples/l5pc

We first load the bluepyopt python module, the ephys submodule and some helper functionality


## install software and data

In [ ]:
 # To avoid problems, install the pacakges you will need.
! pip install --upgrade pip
! pip install json2html
! pip install -q --upgrade "hbp-service-client==1.0.0"

### Restart your kernel now

In [ ]:
!pip install PyYAML==3.10
!pip install mpmath==0.19

In [ ]:
!pip -q install neurom --upgrade

### Unzip the zip directory __.zip__

In [ ]:
extension = '.zip'
file_name = 'cell_optimization_2017' # folder name
full_name = file_name + extension
clients = get_hbp_service_client()
collab_path = get_collab_storage_path()
data_dir = collab_path + '/%s' % full_name
clients.storage.download_file(data_dir, full_name)

In [ ]:
# Let's decompress the folder
!unzip -q {full_name} -d {'cell_optimization_2017'}

In [ ]:
# Let's see what is inside the folder
!ls

In [ ]:
# For some reason when we unzip a zip file the unzip function creates another folder inside
# the unzip folder,so we have to take into account that the folder path has one more step 
# in the path to the desired file
!ls cell_optimization_2017/cell_optimization_2017/

In [ ]:
# compile the mod files
!nrnivmodl cell_optimization/mechanisms/

In [ ]:
%load_ext autoreload
%autoreload
import getpass
import os.path
from __future__ import print_function
import pprint
import bluepyopt
import bluepyopt.ephys as ephys
import bluepyopt as bpopt
pp = pprint.PrettyPrinter(indent=2)

Enable the code below to enable debug level logging

In [ ]:
working_directory = 'cell_optimization_2017/cell_optimization_2017/tmp/model/neoctx-L5pc'

## Model description

### Morphology

We're using a complex reconstructed morphology of an L5PC cell. Let's visualise this with the BlueBrain NeuroM software:

In [ ]:
import neurom.viewer

neurom.viewer.draw(neurom.load_neuron(os.path.join(working_directory, 'morphology', 'C060114A7.asc')))


To load the morphology we create a NrnFileMorphology object. We set 'do_replace_axon' to True to replace the axon with a Axon Initial Segment.

In [ ]:
morphology_filename = working_directory +'/morphology/C060114A7.asc'
morphology = ephys.morphologies.NrnFileMorphology(morphology_filename, do_replace_axon=True)

print(str(morphology))

### Parameters

Since we have many parameters in this model, they are stored in a json file: https://github.com/BlueBrain/BluePyOpt/blob/master/examples/l5pc/config/parameters.json

In [ ]:
# Example of json entry: Frozen Parameter for the equilibrum membrane potential
{
    "param_name": "e_pas",
    "sectionlist": "all",
    "type": "section",
    "dist_type": "uniform",
    "value": -75
},
# Example of json entry: Bounded Parameter
{
    "param_name": "gNaTs2_tbar_NaTs2_t",
    "mech": "NaTs2_t",
    "bounds": [
        0,
        1
    ],
    "dist_type": "uniform",
    "mech_param": "gNaTs2_tbar",
    "type": "range",
    "sectionlist": "somatic"
},

In [ ]:
import json
param_configs = json.load(open(os.path.join(working_directory, 'config.orig/parameters.json')))

print(param_configs)

In [ ]:
# Let's write them as a table

print('{0:<25}: {1:<10}: {2:<15}: {3:<10}: {4:<10}'.format('Parameter Name',
                                                           'Par Type',
                                                           'Value',
                                                           'SecList',
                                                           'Distribution Type'))
print('-'*100)
for param_config in param_configs:
    
    string = '{0:<25}: {1:<10}: '.format(param_config['param_name'],
                                         param_config['type'])

    if 'value' in param_config:
        string += '{0:<15}: '.format(param_config['value'])
    elif 'range' in param_config['type']:
        string += '{0:<15}: '.format(param_config['bounds'])

    if 'sectionlist' in param_config:
        string += '{0:<10}: '.format(param_config['sectionlist'])
    else:
        string += '{0:<10}: '.format('NA')

    if 'dist_type' in param_config:
        
        string += '{0:<10}'.format(param_config['dist_type'])
    else:
        string += '{0:<10}'.format('NA')

    print(string)

As you can see there are two types of parameters, parameters __with a fixed value__ and parameters __with bounds__. The latter will be optimised by the algorithm.

Now we define a function that will load all the parameters in BluePyOpt Parameter objects

In [ ]:
config_dir = working_directory + '/config.orig'

In [ ]:
import os

def define_parameters():
    """Define parameters"""

    param_configs = json.load(open(os.path.join(config_dir, 'parameters.json')))
    parameters = []

    for param_config in param_configs:
        if 'value' in param_config:
            frozen = True
            value = param_config['value']
            bounds = None
        elif 'bounds' in param_config:
            frozen = False
            bounds = param_config['bounds']
            value = None
        else:
            raise Exception(
                'Parameter config has to have bounds or value: %s'
                % param_config)

        if param_config['type'] == 'global':
            parameters.append(
                ephys.parameters.NrnGlobalParameter(
                    name=param_config['param_name'],
                    param_name=param_config['param_name'],
                    frozen=frozen,
                    bounds=bounds,
                    value=value))
        elif param_config['type'] in ['section', 'range']:
            if param_config['dist_type'] == 'uniform':
                scaler = ephys.parameterscalers.NrnSegmentLinearScaler()
            elif param_config['dist_type'] == 'exp':
                scaler = ephys.parameterscalers.NrnSegmentSomaDistanceScaler(
                    distribution=param_config['dist'])
            seclist_loc = ephys.locations.NrnSeclistLocation(
                param_config['sectionlist'],
                seclist_name=param_config['sectionlist'])

            name = '%s.%s' % (param_config['param_name'],
                              param_config['sectionlist'])

            if param_config['type'] == 'section':
                parameters.append(
                    ephys.parameters.NrnSectionParameter(
                        name=name,
                        param_name=param_config['param_name'],
                        value_scaler=scaler,
                        value=value,
                        frozen=frozen,
                        bounds=bounds,
                        locations=[seclist_loc]))
            elif param_config['type'] == 'range':
                parameters.append(
                    ephys.parameters.NrnRangeParameter(
                        name=name,
                        param_name=param_config['param_name'],
                        value_scaler=scaler,
                        value=value,
                        frozen=frozen,
                        bounds=bounds,
                        locations=[seclist_loc]))
        else:
            raise Exception(
                'Param config type has to be global, section or range: %s' %
                param_config)

    return parameters

In [ ]:
parameters = define_parameters()

for p in parameters:
    print('{0:<50}: {1:<10}'.format(p.name, p.value if p.frozen else p.bounds))

## Mechanism

We also need to add all the necessary mechanisms, like ion channels to the model. The configuration of the mechanisms is also stored in a json file, and can be loaded in a similar way.

In [ ]:
def define_mechanisms():
    """Define mechanisms"""

    mech_definitions = json.load(
        open(
            os.path.join(
                config_dir,
                'mechanisms.json')))

    mechanisms = []
    for sectionlist, channels in mech_definitions.items():
        seclist_loc = ephys.locations.NrnSeclistLocation(
            sectionlist,
            seclist_name=sectionlist)
        for channel in channels:
            mechanisms.append(ephys.mechanisms.NrnMODMechanism(
                name='%s.%s' % (channel, sectionlist),
                mod_path=None,
                suffix=channel,
                locations=[seclist_loc],
                preloaded=True))

    return mechanisms

In [ ]:
mechanisms = define_mechanisms()

print('\n'.join(map(str, mechanisms)))

# Cell model

With the morphology, mechanisms and parameters we can build the cell model

In [ ]:
l5pc_cell = ephys.models.CellModel('l5pc', morph=morphology, mechs=mechanisms, params=parameters)
print(l5pc_cell)

For use in the cell evaluator later, we need to make a list of the name of the parameters we are going to optimise. These are the parameters that are not frozen.

In [ ]:
param_names = [param.name for param in l5pc_cell.params.values() if not param.frozen]   

## Protocols

Now that we have a cell model, we can apply protocols to it. The protocols are also stored in a json file.

In [ ]:
proto_configs = json.load(open(os.path.join(config_dir, 'protocols.json')))

for key, value in proto_configs.items():
    print(key)
    for stimulus in value['stimuli']:
        print('\t', stimulus)

And they can be automatically loaded

In [ ]:
def define_protocols():
    """Define protocols"""

    protocol_definitions = json.load(
        open(
            os.path.join(
                config_dir,
                'protocols.json')))

    protocols = {}

    soma_loc = ephys.locations.NrnSeclistCompLocation(
        name='soma',
        seclist_name='somatic',
        sec_index=0,
        comp_x=0.5)

    for protocol_name, protocol_definition in protocol_definitions.items():
        # By default include somatic recording
        somav_recording = ephys.recordings.CompRecording(
            name='%s.soma.v' %
            protocol_name,
            location=soma_loc,
            variable='v')

        recordings = [somav_recording]

        if 'extra_recordings' in protocol_definition:
            for recording_definition in protocol_definition['extra_recordings']:
                if recording_definition['type'] == 'somadistance':
                    location = ephys.locations.NrnSomaDistanceCompLocation(
                        name=recording_definition['name'],
                        soma_distance=recording_definition['somadistance'],
                        seclist_name=recording_definition['seclist_name'])
                    var = recording_definition['var']
                    recording = ephys.recordings.CompRecording(
                        name='%s.%s.%s' % (protocol_name, location.name, var),
                        location=location,
                        variable=recording_definition['var'])

                    recordings.append(recording)
                else:
                    raise Exception(
                        'Recording type %s not supported' %
                        recording_definition['type'])

        stimuli = []
        for stimulus_definition in protocol_definition['stimuli']:
            stimuli.append(ephys.stimuli.NrnSquarePulse(
                step_amplitude=stimulus_definition['amp'],
                step_delay=stimulus_definition['delay'],
                step_duration=stimulus_definition['duration'],
                location=soma_loc,
                total_duration=stimulus_definition['totduration']))

        protocols[protocol_name] = ephys.protocols.SweepProtocol(
            protocol_name,
            stimuli,
            recordings)

    return protocols

In [ ]:
fitness_protocols = define_protocols()
print('\n'.join('%s' % protocol for protocol in fitness_protocols.values()))

## eFeatures

For every protocol we need to define which eFeatures will be used as objectives of the optimisation algorithm.

In [ ]:
feature_configs = json.load(open(os.path.join(config_dir,'features.json')))
pp.pprint(feature_configs)

In [ ]:
def define_fitness_calculator(protocols):
    """Define fitness calculator"""

    feature_definitions = json.load(
        open(
            os.path.join(
                config_dir,
                'features.json')))

    # TODO: add bAP stimulus
    objectives = []

    for protocol_name, locations in feature_definitions.items():
        for location, features in locations.items():
            for efel_feature_name, meanstd in features.items():
                feature_name = '%s.%s.%s' % (
                    protocol_name, location, efel_feature_name)
                recording_names = {'': '%s.%s.v' % (protocol_name, location)}
                stimulus = protocols[protocol_name].stimuli[0]

                stim_start = stimulus.step_delay

                if location == 'soma':
                    threshold = -20
                elif 'dend' in location:
                    threshold = -55

                if protocol_name == 'bAP':
                    stim_end = stimulus.total_duration
                else:
                    stim_end = stimulus.step_delay + stimulus.step_duration

                feature = ephys.efeatures.eFELFeature(
                    feature_name,
                    efel_feature_name=efel_feature_name,
                    recording_names=recording_names,
                    stim_start=stim_start,
                    stim_end=stim_end,
                    exp_mean=meanstd[0],
                    exp_std=meanstd[1],
                    threshold=threshold)
                objective = ephys.objectives.SingletonObjective(
                    feature_name,
                    feature)
                objectives.append(objective)

    fitcalc = ephys.objectivescalculators.ObjectivesCalculator(objectives)

    return fitcalc

In [ ]:
fitness_calculator = define_fitness_calculator(fitness_protocols)
print(fitness_calculator)

## Simulator

We need to define which simulator we will use. In this case it will be Neuron, i.e. the NrnSimulator class

In [ ]:
sim = ephys.simulators.NrnSimulator()

## Evaluator

With all the components defined above we can build a cell evaluator

In [ ]:
evaluator = ephys.evaluators.CellEvaluator(                                          
        cell_model=l5pc_cell,                                                       
        param_names=param_names,                                                    
        fitness_protocols=fitness_protocols,                                        
        fitness_calculator=fitness_calculator,                                      
        sim=sim)  

This evaluator can be used to run the protocols. The original parameter values for the Markram et al. 2015 L5PC model are:

In [ ]:
release_params = {
    'gamma_CaDynamics_E2.somatic': 0.000609,
    'gSKv3_1bar_SKv3_1.somatic': 0.303472,
    'gSK_E2bar_SK_E2.somatic': 0.008407,
    'gCa_HVAbar_Ca_HVA.somatic': 0.000994,
    'gNaTs2_tbar_NaTs2_t.somatic': 0.983955,
    'decay_CaDynamics_E2.somatic': 210.485284,
    'gCa_LVAstbar_Ca_LVAst.somatic': 0.000333
}

Running the responses is as easy as passing the protocols and parameters to the evaluator. (The line below will take some time to execute)

In [ ]:
release_responses = evaluator.run_protocols(protocols=fitness_protocols.values(), param_values=release_params)

We can now plot all the responses

In [ ]:
import matplotlib.pyplot as plt

def plot_responses(responses):
    fig, axes = plt.subplots(len(responses), figsize=(10,10))
    for index, (resp_name, response) in enumerate(sorted(responses.items())):
        axes[index].plot(response['time'], response['voltage'], label=resp_name)
        axes[index].set_title(resp_name)
    fig.tight_layout()
    fig.show()
plot_responses(release_responses)

Running an optimisation of the parameters now has become very easy. Of course running the L5PC optimisation will require quite some computing resources.

To show a proof-of-concept, we will only run 2 generations, with 2 offspring individuals per generations. If you want to run all full optimisation, you should run for 100 generations with an offspring size of 100 individuals. However, 
__don't run more than 10 generations__, because it can colaps the collab. You will see that already with 2 iterations 
the process takes several minutes.


In [ ]:
opt = bpopt.optimisations.DEAPOptimisation(evaluator=evaluator, offspring_size=10)

final_pop, halloffame, log, hist = opt.run(max_ngen=2, cp_filename='cell_optimization/checkpoints/checkpoint.pkl')

'''
import pickle

cp = pickle.load(open('checkpoints/checkpoint.pkl'))

final_pop = cp['generation']
hist = cp['history']
halloffame = cp['halloffame']
log = cp['logbook']
'''

The first individual in the hall of fame will contain the best solution found.

In [ ]:
print(halloffame[0])

These are the raw parameter values. The evaluator object can convert this in a dictionary, so that we can see the parameter names corresponding to these values.

In [ ]:
best_params = evaluator.param_dict(halloffame[0])
print(pp.pprint(best_params))

Then we can run the fitness protocols on the model with these parameter values


In [ ]:
best_responses = evaluator.run_protocols(protocols=fitness_protocols.values(), param_values=best_params)

And then we can also plot these responses.

When you ran the above optimisation with only 2 individuals and 2 generations, this 'best' model will of course be very low quality.


In [ ]:
plot_responses(best_responses)

Let's visualize the objectives of the optimisation. First we need to create a dictionary of the responses of the best individual, that is the NEURON simulation of the voltage trace for each protocol with the parameters that correspond to the best individual. Remember that an individual is just a set of our model's parameters.

In [ ]:
fitness_protocols = evaluator.fitness_protocols

responses = {}
nrn = ephys.simulators.NrnSimulator()
for protocol in fitness_protocols.values():
    response = protocol.run(
                            cell_model=opt.evaluator.cell_model,
                            param_values=best_params,
                            sim=nrn
                            )
    responses.update(response)

responses

Plot the objectives of the responses, that is the deviation (in standard deviations) of the features from the values in the input features.json file.

In [ ]:
objectives = opt.evaluator.fitness_calculator.calculate_scores(responses)

f, ax = plt.subplots()

ytick_pos = [x + 0.5 for x in range(len(objectives.keys()))]
ax.barh(ytick_pos,
              objectives.values(),
              height=0.5,
              align='center',
              color='#779ECB')

ax.set_yticks(ytick_pos) 
ax.set_yticklabels(objectives.keys(), size='x-small') 
ax.set_ylim(-0.5, len(objectives.values()) + 0.5) 
ax.set_xlabel('Objective value (# std)') 
ax.set_ylabel('Objectives') 

plt.tight_layout()

Let's now plot the traces for all the individuals in the hall of fame. In other words, for each parameter set = individual let's plot the respective trace.

First we create a list for all the individuals, in a simimilar manner as above, but for all the individuals in the hall of fame this time.

In [ ]:
hof_responses = []
for best_ind in halloffame:
    best_ind_dict = evaluator.param_dict(best_ind)
    hof_responses.append(evaluator.run_protocols(evaluator.fitness_protocols.values(),
                                             param_values=best_ind_dict))

Let's now plot the traces for all the individuals in the hall of fame. In other words, for each parameter set = individual let's plot the respective trace.

First we create a list for all the individuals, in a simimilar manner as above, but for all the individuals in the hall of fame this time.


In [ ]:
hof_responses = []
for best_ind in halloffame:
    best_ind_dict = evaluator.param_dict(best_ind)
    hof_responses.append(evaluator.run_protocols(evaluator.fitness_protocols.values(),
                                             param_values=best_ind_dict))

In [ ]:
traces = sorted(hof_responses[0].keys())

traces

Then we plot the traces with the best one in deep blue

In [ ]:
import numpy as np

def plot_multiple_responses(responses, fig):
    '''creates 6 subplots for step{1,2,3} and dAP traces, and plots all the responses on them'''

    plot_count = len(traces)
    
    # one subplot for each trace
    ax = [fig.add_subplot(plot_count, 1, i + 1) for i in range(plot_count)]

    # to color the last one deep blue
    overlay_count = len(responses)
    
    for index, trace in enumerate(traces):  
        for n, response in enumerate(reversed(responses)): # worst to best individual
            
            color='lightblue' if n < overlay_count - 1 else 'blue'
            
            ax[index].plot(response[trace]['time'],
                           response[trace]['voltage'],
                           color=color,
                           linewidth=1)

        # formatting of the subplots
        ax[index].set_xlabel('Time (ms)')
        ax[index].set_ylabel('Voltage (mV)')
        ax[index].text(0.01, 0.7, trace, transform=ax[index].transAxes, fontsize=10,
        verticalalignment='top')
        #ax[index].set_xlim(80, 200)
            

        
f = plt.figure(figsize=(10,10))
plot_multiple_responses(hof_responses, f)

In [ ]:
responses

And finally the fitness function

In [ ]:
import numpy

logs=log
gen_numbers = log.select('gen')
min_fitness = numpy.array(log.select('min'))
max_fitness = log.select('max')
mean_fitness = numpy.array(log.select('avg'))
std_fitness = numpy.array(log.select('std'))

fig, ax = plt.subplots(1, figsize=(10, 5), facecolor='white')

std = std_fitness
mean = mean_fitness
minimum = min_fitness
stdminus = mean - std                                                           
stdplus = mean + std

ax.plot(                                                                      
    gen_numbers,                                                                
    mean,                                                                       
    color='black',                                                              
    linewidth=2,                                                                
    label='population average')                                                 

ax.fill_between(                                                              
    gen_numbers,                                                                
    stdminus,                                                                   
    stdplus,                                                                    
    color='lightgray',                                                          
    linewidth=2,                                                                
    label=r'population standard deviation')                                     

ax.plot(                                                                      
    gen_numbers,                                                                
    minimum,                                                                    
    color='red',                                                                
    linewidth=2,                                                                
    label='population minimum')                                                 

ax.set_xlim(min(gen_numbers) - 1, max(gen_numbers) + 1)                       
ax.set_xlabel('Generation #')                                                 
ax.set_ylabel('Sum of objectives')                                            
ax.set_ylim([0, max(stdplus)])                                                
ax.legend()                        


fig.show()
